In [1]:
#Training data augmentation
#Written by Lehan Yao from Qian Chen Group, MatSE, UIUC
#Sept. 21, 2021

In [2]:
import cv2
import numpy as np
import glob
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
labelPath = 'manual label'
nameList = [os.path.basename(x) for x in glob.glob(labelPath+'/image/*.tif')]
stack = np.zeros([len(nameList),512,512,3]) #container for storing the labels and images

i = 0
for name in nameList: #read the labels and images and store them in RGB channels
    I_image = cv2.imread(labelPath + '/image/' + name )
    mask = cv2.imread(labelPath + '/mask/' + name)
    stack[i,:,:,0] = cv2.cvtColor(I_image, cv2.COLOR_BGR2GRAY)
    stack[i,:,:,1] = mask[:,:,0]
    stack[i,:,:,2] = mask[:,:,2]
    i += 1

In [9]:
volume = 1000 #the number of images in the augmented dataset
data_gen_args_train = dict(fill_mode = 'reflect', rotation_range=360, zoom_range=0.1, horizontal_flip = True, vertical_flip = True, brightness_range = (0.8,1))
data_gen_args_label = dict(fill_mode = 'reflect', rotation_range=360, zoom_range=0.1, horizontal_flip = True, vertical_flip = True)

datagen_train = ImageDataGenerator(**data_gen_args_train)
datagen_label = ImageDataGenerator(**data_gen_args_label)
seed = 1
image_generator_train = datagen_train.flow(np.expand_dims(stack[:,:,:,0],3),seed=seed)
empty = np.expand_dims(stack[:,:,:,0]*0,3) #since the label only contain two layers, a space holder is added into the 3rd channel
image_generator_label = datagen_label.flow(np.concatenate([empty,stack[:,:,:,1:]],axis = 3),seed=seed)
i = 0
while (i < volume):
    im = next(image_generator_train) # the generators generates multiple images per iteration. But with setting the same seed, only the first image of two generators seems to be the same
    lb = next(image_generator_label) # the generators generates multiple images per iteration. But with setting the same seed, only the first image of two generators seems to be the same
    t = im[0,:,:,0]/255
    m1 = lb[0,:,:,1]/255
    m2 = lb[0,:,:,2]/255
    m3 = m1*0
    m = np.stack([m1,m3,m2],2)*255
    t = t*255
    t = np.uint8(t)
    m = np.uint8(m)
    cv2.imwrite('train/'+str(i)+'.tif', t)
    cv2.imwrite('label/'+str(i)+'.tif', m)
    i += 1